# Legal Long Span Extraction

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Legal/8.Long_Span_Extraction.ipynb)

# Colab Setup

In [4]:
import sys
import json
import os
with open('../../spark_nlp_for_healthcare_spark_ocr_4.0.2.json') as f:
    license_keys = json.load(f)
    
import os
locals().update(license_keys)
os.environ.update(license_keys)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [10]:
import sparknlp
import sparknlp_jsl

print("sparknlp version:",sparknlp.version())
print("sparknlp_jsl version:", sparknlp_jsl.version())

sparknlp version: 4.1.0
sparknlp_jsl version: 4.1.0a6


# Start Spark Session

In [11]:
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

print("sparknlp version:",sparknlp.version())
print("sparknlp_jsl version:", sparknlp_jsl.version())

builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "54G")\
        .config("spark.executor.heartbeatInterval", "60s")\
        .config("spark.executor.memory", "6G")\
        .config("spark.driver.maxResultSize", "8G")\
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+ PUBLIC_VERSION) \
        .config("spark.jars", "/home/jovyan/work/shared/juan/_FINLEGAL/_QAGenerator/spark-nlp-jsl-4.1.0a6.jar")
spark = builder.getOrCreate()

sparknlp version: 4.1.0
sparknlp_jsl version: 4.1.0a6
:: loading settings :: url = jar:file:/home/jovyan/work/shared/venvs/qagenerator/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dfa4631d-ef2d-4cbc-851f-47efe6db3c74;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.0.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lom

In [12]:
spark

# NER, Question Generation and Question Answering for Long-Span extraction

Legal documents are known to be very long. Although you can divide the docuuments into paragraphs or sections, and those into sentences, the resulted sentences are still long.

Let's take a look at this example:

`Buyer shall use such materials and supplies only in accordance with the present agreement`

Not, let's imagine we want to extract three entities:
1) The Subject (`Buyer`)
2) The Action (`shall use`)
3) The Object (what the Buyer shall use? - `such materials and supplies only in accordance with the present agreement`)

Although Subject and Action can be totally manageable by traditional NER, it usually struggles the longer the spans are. Trying to model the extraction of Object with a simple NER may result in word fading, when some of the initial or ending words fade into `O`.

We present in this notebook a solution for Long Span Extraction: Using an Automatic Question Generator and a Question Answering model to:
1) First, using NER, detect entities as the `Subject` and the `Action`. 

Example: `Buyer - SUBJECT`, `shall use - OBJECT`

2) Automatically generate a question to ask for the `Object`, using `Subject` and `Action`;

Example: `What shall the Buyer use?`

3) Use the question and the sentence to retrieve `Object`, without the limitations of traditional NER;

Example: `What shall the Buyer use? such materials and supplies only in accordance with the present agreement`

Last, but not least, it's very important to chose a domain-specific Question Answering model.


# Answering the question - `What?`
Let's suppose we have the sentence of the example:

`The Buyer shall use such materials and supplies only in accordance with the present agreement`

In Spark NLP for Legal, we have a trained NER model which is able to extract Subjects (`Buyer`) and Actions (`shall use`) of agreements / obligations with good accuracy.

It's also trained for extracting the `Object` using NER, but it's usage is limited due to the restrictions commented above.

Let's get SUBJECT and ACTION and automatically create a question with them.

In [ ]:

text = """The Buyer shall use such materials and supplies only in accordance with the present agreement"""

In [41]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sparktokenizer = Tokenizer()\
  .setInputCols("document")\
  .setOutputCol("token")

tokenClassifier = LegalBertForTokenClassification.pretrained("legner_obligations", "en", "legal/models")\
  .setInputCols("token", "document")\
  .setOutputCol("label")\
  .setCaseSensitive(True)

nerconverter = NerConverter()\
  .setInputCols(["document", "token", "label"])\
  .setOutputCol("ner_chunk")

# setEntities1 says which entity from NER goes first in the question
# setEntities2 says which entity from NER goes second in the question
# setQuestionMark to True adds a '?' at the end of the sentence (after entity 2)
# To sum up, the pattern is     [QUESTIONPRONOUN] [ENTITY1] [ENTITY2] [QUESTIONMARK]
qagenerator = NerQAGenerator()\
  .setInputCols(["ner_chunk"])\
  .setOutputCol("question")\
  .setQuestionMark(False)\
  .setQuestionPronoun("What")\
  .setEntities1(["OBLIGATION_SUBJECT"])\
  .setEntities2(["OBLIGATION_ACTION"])

qa = RoBertaForQuestionAnswering.pretrained("legqa_roberta","en", "legal/models") \
  .setInputCols(["question", "document"]) \
  .setOutputCol("answer") \
  .setCaseSensitive(True)
  
pipeline =  Pipeline(stages=[
  documentAssembler,
  sparktokenizer,
  tokenClassifier,
  nerconverter,
  qagenerator,
    qa
    ]
)

p_model = pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

res = p_model.transform(spark.createDataFrame([[text]]).toDF("text"))

albert_qa_xxlargev1_squad2_512 download started this may take some time.
Approximate size to download 736.4 MB
[OK!]


In [43]:
res.select('question.result').show(truncate=False)

+-----------------------+
|result                 |
+-----------------------+
|[What Buyer shall use ]|
+-----------------------+



In [44]:
res.select('answer.result').show(truncate=False)

+---------------------------------------------------------------------------+
|result                                                                     |
+---------------------------------------------------------------------------+
|[such materials and supplies only in accordance with the present agreement]|
+---------------------------------------------------------------------------+



Let's get 4 additional examples

In [45]:
texts = [
    """The Buyer shall use such materials and supplies only in accordance with the present agreement""",
    """The Provider will notify the Buyer about the release date""",
    """Amazon agrees to supply 1-year license without fees""",
    """The Supplier should ship the product in less than 1 month"""
]

pdf = pd.DataFrame(texts, columns = ["text"])
pdf

,text
0,The Buyer shall use such materials and supplie...
1,The Provider will notify the Buyer about the r...
2,Amazon agrees to supply 1-year license without...
3,The Supplier should ship the product in less t...


In [ ]:
df = spark.createDataFrame(pdf)

In [ ]:
res = p_model.transform(df)

In [48]:
res.select('question.result', 'answer.result').show(truncate=False)

+-------------------------------+---------------------------------------------------------------------------+
|result                         |result                                                                     |
+-------------------------------+---------------------------------------------------------------------------+
|[What Buyer shall use ]        |[such materials and supplies only in accordance with the present agreement]|
|[What Provider will notify ]   |[theBuyer about the release date]                                          |
|[What Amazon agrees to supply ]|[1-year license without fees]                                              |
|[What Supplier should ship ]   |[the product in less than 1 month]                                         |
+-------------------------------+---------------------------------------------------------------------------+



# Answering the question - `To whom?`

Let's try to get now the Indirect Object. That is, the recipient of an action. For example, to whom a supplier should send a shipment.

In [49]:
qagenerator = NerQAGenerator()\
  .setInputCols(["ner_chunk"])\
  .setOutputCol("question")\
  .setQuestionMark(False)\
  .setQuestionPronoun("To whom")\
  .setEntities1(["OBLIGATION_ACTION"])\
  .setEntities2(["OBLIGATION_SUBJECT"])
  
pipeline =  Pipeline(stages=[
  documentAssembler,
  sparktokenizer,
  tokenClassifier,
  nerconverter,
  qagenerator,
    qa
    ]
)

p_model = pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

text = """The Provider shall send the shipment to the Buyer"""
res = p_model.transform(spark.createDataFrame([[text]]).toDF("text"))

albert_qa_xxlargev1_squad2_512 download started this may take some time.
Approximate size to download 736.4 MB
[OK!]


In [51]:
res.select('text', 'question.result', 'answer.result').show(truncate=False)

+-------------------------------------------------+------------------------------+----------+
|text                                             |result                        |result    |
+-------------------------------------------------+------------------------------+----------+
|The Provider shall send the shipment to the Buyer|[To whom shall send Provider ]|[theBuyer]|
+-------------------------------------------------+------------------------------+----------+



# Other clauses
This approach works very well also with other clauses and phrases, as temporal ones. Let's try to ask for the deadline of a contract

In [54]:
qagenerator = NerQAGenerator()\
  .setInputCols(["ner_chunk"])\
  .setOutputCol("question")\
  .setQuestionMark(False)\
  .setQuestionPronoun("Before when")\
  .setEntities1(["OBLIGATION_ACTION"])\
  .setEntities2(["OBLIGATION_SUBJECT"])
  
pipeline =  Pipeline(stages=[
  documentAssembler,
  sparktokenizer,
  tokenClassifier,
  nerconverter,
  qagenerator,
    qa
    ]
)

p_model = pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

text = """The customer should sign the contract before May, 2023"""
res = p_model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [55]:
res.select('text', 'question.result', 'answer.result').show(truncate=False)

+------------------------------------------------------+-----------------------------------+-----------------+
|text                                                  |result                             |result           |
+------------------------------------------------------+-----------------------------------+-----------------+
|The customer should sign the contract before May, 2023|[Before when should sign customer ]|[beforeMay, 2023]|
+------------------------------------------------------+-----------------------------------+-----------------+

